In [171]:
from typing import Any
from torch.utils.data import DataLoader, Dataset
from config import *
import polars as pl
import os
import torch
import torch.nn.functional as F
from data_process_pl import make_label, target_car_dict, get_cycle_charge
import matplotlib.pyplot as plt
from data_process_pl import *

In [2]:
class processor:
    def __init__(self, groups, seq_len, step_size) -> None:
        self.groups = groups
        self.seq_len = seq_len
        self.step_size = step_size
    
    def process_data(self, data:pl.DataFrame):
        final_res = []
        for item in data.groupby(self.groups):
            data_tmp = item[1]
            l = data_tmp.shape[0]
            if l < seq_len:
                continue
            steps = int((l-self.seq_len)//self.step_size+1)
            for i in range(steps):
                X_tensor = torch.from_numpy(data_tmp[self.step_size*i:self.step_size*i+self.seq_len].to_numpy())
                final_res.append(X_tensor)
            X_tensor = torch.from_numpy(data_tmp[-self.seq_len:].to_numpy())
            final_res.append(X_tensor)
        ares = torch.cat(final_res)
        ares = ares.reshape((-1, self.seq_len, data.shape[1]))
        return ares

class BatteryData(Dataset):
    def __init__(self, data_dir, processor) -> None:
        super().__init__()
        self.processor = processor
        self.pl_data = pl.read_parquet(data_dir)

    def __len__(self):
        return self.pl_data.shape[0]
    
    def __getitem__(self, index: Any) -> Any:
        ares = self.processor.process_data()
        X = ares[:, :, 3:]
        Y = ares[:, :, 2]
        return X[index], Y[index]
    
    @staticmethod
    def get_data_info(data_dir):
        data_info={}
        for root, dirs, files in os.walk(data_dir):
            for file in files:
                file_path = os.path.join(root, file)
                print(f"file: {file}")
                data = pl.read_parquet(file_path)
                yield data
    

In [3]:
df_charge = pl.read_parquet('data/D3_4_5.parquet')

In [4]:
def get_charge_data(car_no):
    df_charge = pl.read_parquet(f"data/{car_no}_4_5.parquet")
    df_charge = df_charge.with_columns(change=pl.col("D24").diff())
    df_charge = df_charge.filter(pl.col("change")<1)
    df_charge = df_charge.sort("samplingtimesec")
    df_charge = get_cycle_charge(df_charge)
    return df_charge

In [5]:
df_charge = get_charge_data("D3")

In [6]:
df_charge = make_label(df_charge, target_car_dict['D3'])

In [7]:
def preprocess(df):
    df = df.with_columns(pl.col('D24').floor().alias("D24_int"))
    df = df.with_columns(soc_diff = pl.col('D40')-pl.col('D41'))
    df = df.with_columns([pl.col(f"D135_{i}").cast(pl.Float32) for i in range(1,113)])
    df = df.with_columns([pl.col(col_name).cast(pl.Int32) for col_name in ['cycle','D24_int','label','D27']])
    df = df.with_columns([pl.col(f"D136_{i}").cast(pl.Float32) for i in range(1,29)])
    df = df.with_columns([pl.col(col_name).cast(pl.Float32) for col_name in ['D27','soc_diff','D22','D356','D52','D334','D290','D133']])
    df = df.with_columns([pl.col(f"D14{i}").cast(pl.Float32) for i in range(2)])
    return df

In [8]:
df_charge = preprocess(df_charge)

In [ ]:
# numerical： D135, D136, soc_diff, D22电池电流, D356(drop), D52低电压, D334内部压力, D24SOC, D40,D41,D44均衡时间,D72开关次数,D117环境温度,D133里程
# cat: D27电池均衡状态, D19状态

In [46]:
df_charge.head()

shape: (5, 172)
┌─────────────────┬──────┬───────┬─────┬───┬───────┬───────┬─────────┬──────────┐
│ samplingtimesec ┆ D140 ┆ D141  ┆ D19 ┆ … ┆ cycle ┆ label ┆ D24_int ┆ soc_diff │
│ ---             ┆ ---  ┆ ---   ┆ --- ┆   ┆ ---   ┆ ---   ┆ ---     ┆ ---      │
│ i64             ┆ f32  ┆ f32   ┆ i64 ┆   ┆ i32   ┆ i32   ┆ i32     ┆ f32      │
╞═════════════════╪══════╪═══════╪═════╪═══╪═══════╪═══════╪═════════╪══════════╡
│ 1630677745000   ┆ 0.0  ┆ 421.0 ┆ 4   ┆ … ┆ 1     ┆ 0     ┆ 60      ┆ 1.5      │
│ 1630677746000   ┆ 0.0  ┆ 421.0 ┆ 4   ┆ … ┆ 1     ┆ 0     ┆ 60      ┆ 1.5      │
│ 1630677747000   ┆ 0.0  ┆ 422.0 ┆ 4   ┆ … ┆ 1     ┆ 0     ┆ 60      ┆ 1.5      │
│ 1630677748000   ┆ 0.0  ┆ 422.0 ┆ 4   ┆ … ┆ 1     ┆ 0     ┆ 60      ┆ 1.5      │
│ 1630677749000   ┆ 0.0  ┆ 422.0 ┆ 4   ┆ … ┆ 1     ┆ 0     ┆ 60      ┆ 1.5      │
└─────────────────┴──────┴───────┴─────┴───┴───────┴───────┴─────────┴──────────┘

In [9]:
final_cols=[f"D135_{i}" for i in range(1,113)]+[f"D136_{i}" for i in range(1,29)]

In [10]:
final_cols = final_cols+["soc_diff",'D22','D52','D24','D40','D41','D44','D72','D117','D133','cycle','D140','D141','label']

In [ ]:
final_cols

In [11]:
df_charge = df_charge.select(final_cols)

In [25]:
df_charge.write_parquet('data/sampling_data.parquet')

In [53]:
def process_data_sample(target_car_dict):
    # for item in normal_car_no:
    #     target_car_dict[item]=''
    final_cols=[f"D135_{i}" for i in range(1,113)]+[f"D136_{i}" for i in range(1,29)]+["soc_diff",'D22','D52','D24','D40','D41','D44','D72','D117','D133','cycle','D140','D141','label']
    for car, target_dt in target_car_dict.items():
        print(car)
        df_charge = get_charge_data(car)
        df_charge = df_charge.with_columns([pl.col(col_name).cast(pl.Int32) for col_name in ['D140','D141']])
        df_charge = make_label(df_charge, target_dt)
        df_charge = preprocess(df_charge)
        df_charge = df_charge.select(final_cols)
        df_charge.write_parquet(f"processed_data/{car}_4_5.parquet")
        del df_charge
        df_discharge = get_discharge_data(car)
        df_discharge = df_discharge.with_columns([pl.col(col_name).cast(pl.Int32) for col_name in ['D140','D141']])
        df_discharge = make_label(df_discharge, target_dt)
        df_discharge = preprocess(df_discharge)
        df_discharge = df_discharge.select(final_cols)
        df_discharge.write_parquet(f"processed_data/{car}_3.parquet")
    return

In [50]:
target_car_dict

{'D3': '2022-07-28',
 'D2': '2022-05-20',
 'D4': '2022-07-25',
 'D1': '2022-03-14',
 'D13': '2022-08-03',
 'D105': '',
 'D280': '',
 'D107': '',
 'D119': '',
 'D180': '',
 'D131': '',
 'D246': '',
 'D162': '',
 'D296': ''}

In [29]:
for item in ['D440','D441','D442','D443']:
    del target_car_dict[item]

In [51]:
for item in ['D105','D107','D119','D131','D162']:
    del target_car_dict[item]

In [39]:
normal_car_no = ['D105','D280','D107','D119','D180','D131','D246','D162','D296']

In [54]:
process_data_sample(target_car_dict)

D3
D2
D4
D1
D13
D280
D180
D246
D296


In [123]:
df_discharge = get_discharge_data('D1')

In [124]:
def make_label(df_charge,target_dt=''):
    if target_dt != '':
        df_tmp = df_charge.groupby("cycle").agg(D24_range=(pl.col('D24').max()-pl.col('D24').min()),
                              startTime=pl.col('samplingtimesec').min(),
                              endTime=pl.col('samplingtimesec').max()).sort('D24_range',descending=True)
        target_dt = time.strptime(target_dt, "%Y-%m-%d")
        target_dt = time.mktime(target_dt)*1000
        low_cycle = df_tmp.filter((target_dt - pl.col('endTime'))<=0)['cycle'].min()-10
        up_cycle = df_tmp.filter((target_dt - pl.col('endTime'))<=0)['cycle'].max()+5
        df_charge=df_charge.with_columns(pl.col("cycle").apply(lambda x: 1 if x>=low_cycle and x<= up_cycle else 0).alias("label"))
    else:
        df_charge=df_charge.select(pl.lit(0).alias('label'),pl.all())
    return df_charge

In [125]:
df_discharge = make_label(df_discharge, '2022-03-14')

In [132]:
df_discharge = preprocess(df_discharge)
df_discharge = df_discharge.with_columns([pl.col(col_name).cast(pl.Int32) for col_name in ['D140','D141']])

In [136]:
avg_cols = ['D141','D22','D24_int','D44','D40','D41','D52','D117','D133','D334','D356','D290','D135_diff','D136_diff','soc_diff']+[f"D135_{i}" for i in range(1,113)]+[f"D136_{i}" for i in range(1,29)]
sum_cols=['D27']
max_cols=['D135_diff','D135_std','D136_diff','D136_std','soc_diff']
final_cols=avg_cols+sum_cols+[f"{item}_max" for item in max_cols]+[f"D135_{i}_dv" for i in range(1,113)]+['cycle']

In [153]:
res_data = df_discharge.groupby(['cycle','D24_int']).apply(process_data)

In [142]:
res_data.filter((pl.col('cycle')==1))

(60, 274)

In [ ]:
def check_data(df):
    print(f"cycle: {df['cycle'][0]}")
    print(f"D24_int: {df['D24_int'][0]}")
    return df
df_charge.groupby(['cycle','D24_int']).apply(check_data)

In [75]:
temp = temp.with_columns([(pl.col(f'D135_{i}').last()-pl.col(f'D135_{i}').first()).alias(f"D135_{i}_dv") for i in range(1,113)])[0]

shape: (1, 284)
┌─────────────────┬──────┬──────┬─────┬───┬─────────────┬─────────────┬─────────────┬─────────────┐
│ samplingtimesec ┆ D140 ┆ D141 ┆ D19 ┆ … ┆ D135_109_dv ┆ D135_110_dv ┆ D135_111_dv ┆ D135_112_dv │
│ ---             ┆ ---  ┆ ---  ┆ --- ┆   ┆ ---         ┆ ---         ┆ ---         ┆ ---         │
│ i64             ┆ str  ┆ str  ┆ i64 ┆   ┆ f32         ┆ f32         ┆ f32         ┆ f32         │
╞═════════════════╪══════╪══════╪═════╪═══╪═════════════╪═════════════╪═════════════╪═════════════╡
│ 1634144499000   ┆ 0    ┆ 426  ┆ 5   ┆ … ┆ 14.0        ┆ 14.0        ┆ 13.0        ┆ 14.0        │
└─────────────────┴──────┴──────┴─────┴───┴─────────────┴─────────────┴─────────────┴─────────────┘

In [ ]:
df_charge.filter(pl.col('D72')==10400).head(20)

In [ ]:
df_charge.columns

### 需要avg的字段：
['D141','D22','D24','D44','D40','D41','D52','D117','D133','D334','D356','D290','D135_diff','D136_diff','soc_diff']
+[f"D135_{i} for i in range(1,113)]+[f"D136_{i} for i in range(1,29)]
### 需要sum的字段
['D27']
### 需要max的字段
['D135_diff','D135_std','D136_diff','D136_std','soc_diff']

In [103]:
temp.with_columns([pl.col(item).mean() for item in avg_cols]+
                 [pl.col(item).max().alias(f"{item}_max") for item in max_cols]+
                 [(pl.col(f'D135_{i}').last()-pl.col(f'D135_{i}').first()).alias(f"D135_{i}_dv") for i in range(1,113)]+
                 [pl.col('D27').sum()]).select(final_cols)[0]

shape: (1, 273)
┌────────────┬────────────┬───────────┬─────┬───┬────────────┬────────────┬────────────┬───────────┐
│ D141       ┆ D22        ┆ D24       ┆ D44 ┆ … ┆ D135_109_d ┆ D135_110_d ┆ D135_111_d ┆ D135_112_ │
│ ---        ┆ ---        ┆ ---       ┆ --- ┆   ┆ v          ┆ v          ┆ v          ┆ dv        │
│ f64        ┆ f32        ┆ f64       ┆ f64 ┆   ┆ ---        ┆ ---        ┆ ---        ┆ ---       │
│            ┆            ┆           ┆     ┆   ┆ f32        ┆ f32        ┆ f32        ┆ f32       │
╞════════════╪════════════╪═══════════╪═════╪═══╪════════════╪════════════╪════════════╪═══════════╡
│ 426.369146 ┆ -22.423691 ┆ 55.450689 ┆ 0.0 ┆ … ┆ 14.0       ┆ 14.0       ┆ 13.0       ┆ 14.0      │
└────────────┴────────────┴───────────┴─────┴───┴────────────┴────────────┴────────────┴───────────┘

In [189]:
def process_data_sample(target_car_dict):
    # for item in normal_car_no:
    #     target_car_dict[item]=''
    avg_cols = ['D141','D22','D24_int','D44','D40','D41','D52','D117','D133','D334','D356','D290','D135_diff','D136_diff','soc_diff']+[f"D135_{i}" for i in range(1,113)]+[f"D136_{i}" for i in range(1,29)]
    sum_cols=['D27']
    max_cols=['D135_diff','D135_std','D136_diff','D136_std','soc_diff']
    final_cols=avg_cols+sum_cols+[f"{item}_max" for item in max_cols]+[f"D135_{i}_dv" for i in range(1,113)]+['label','cycle','samplingtimesec']
    def process_data(df):
        nonlocal final_cols
        temp = df.with_columns([pl.col(item).mean() for item in avg_cols]+
                     [pl.col(item).max().alias(f"{item}_max") for item in max_cols]+
                     [(pl.col(f'D135_{i}').last()-pl.col(f'D135_{i}').first()).alias(f"D135_{i}_dv") for i in range(1,113)]+
                     [pl.col('samplingtimesec').min()]+
                     [pl.col('D27').sum()]).select(final_cols)[0]
        return temp
    for car, target_dt in target_car_dict.items():
        print(car)
        df_charge = get_charge_data(car)
        df_charge = df_charge.with_columns([pl.col(col_name).cast(pl.Int32) for col_name in ['D140','D141']])
        df_charge = make_label(df_charge, target_dt)
        df_charge = preprocess(df_charge)
        df_charge = df_charge.groupby(['cycle','D24_int']).apply(process_data)
        df_charge = df_charge.with_columns([pl.col(item).fill_null(0) for item in ['D334','D356','D290']])
        df_charge = df_charge.filter(pl.col('cycle').is_null()==False)
        df_charge = df_charge.sort(['cycle','D24_int'], descending=False)
        if target_dt == '':
            df_charge.write_parquet(f"processed_data/normal_sample/charge_data/{car}_4_5.parquet")
        else:
            df_charge.write_parquet(f"processed_data/neg_sample/charge_data/{car}_4_5.parquet")
        del df_charge
        df_discharge = get_discharge_data(car)
        df_discharge = df_discharge.with_columns([pl.col(col_name).cast(pl.Int32) for col_name in ['D140','D141']])
        df_discharge = make_label(df_discharge, target_dt)
        df_discharge = preprocess(df_discharge)
        df_discharge = df_discharge.groupby(['cycle','D24_int']).apply(process_data)
        df_discharge = df_discharge.with_columns([pl.col(item).fill_null(0) for item in ['D334','D356','D290']])
        df_discharge = df_discharge.filter(pl.col('cycle').is_null()==False)
        df_discharge = df_discharge.sort(['cycle','D24_int'], descending=False)
        if target_dt=='':
            df_discharge.write_parquet(f"processed_data/normal_sample/discharge_data/{car}_3.parquet")
        else:
            df_discharge.write_parquet(f"processed_data/neg_sample/discharge_data/{car}_3.parquet")
    return

In [191]:
target_car_dict = {'D119':''}

In [187]:
target_car_dict = {'D3':'2022-07-28',
                   'D2':'2022-05-20',
                   'D4':'2022-07-25',
                      'D1':'2022-03-14',
                      'D13':'2022-08-03',
                      'D440':'2022-09-06',
                      'D441':'2022-09-13',
                      'D442':'2022-11-21',
                      'D443':'2022-11-17',
                     'D105': '',
                     'D280': '',
                     'D107': '',
                     'D180': '',
                     'D131': '',
                     'D246': '',
                     'D162': '',
                     'D296': ''}

In [190]:
process_data_sample(target_car_dict)

D3
D2
D4
D1
D13
D440
D441
D442
D443
D105
D280
D107
D180
D131
D246
D162
D296


In [152]:
print(res_data.shape)
res_data=res_data.drop_nulls()
res_data = res_data.sort(['cycle','D24_int'], descending=False)
print(res_data.shape)

(4783, 274)
(2474, 274)


In [155]:
res_data.select(pl.all().is_null().sum())

shape: (1, 274)
┌──────┬─────┬─────────┬─────┬───┬─────────────┬─────────────┬─────────────┬───────┐
│ D141 ┆ D22 ┆ D24_int ┆ D44 ┆ … ┆ D135_110_dv ┆ D135_111_dv ┆ D135_112_dv ┆ cycle │
│ ---  ┆ --- ┆ ---     ┆ --- ┆   ┆ ---         ┆ ---         ┆ ---         ┆ ---   │
│ u32  ┆ u32 ┆ u32     ┆ u32 ┆   ┆ u32         ┆ u32         ┆ u32         ┆ u32   │
╞══════╪═════╪═════════╪═════╪═══╪═════════════╪═════════════╪═════════════╪═══════╡
│ 0    ┆ 0   ┆ 0       ┆ 0   ┆ … ┆ 0           ┆ 0           ┆ 0           ┆ 1     │
└──────┴─────┴─────────┴─────┴───┴─────────────┴─────────────┴─────────────┴───────┘

In [160]:
res_data = res_data.with_columns([pl.col(item).fill_null(0) for item in ['D334','D356','D290']])
res_data = res_data.filter(pl.col('cycle').is_null()==False)
res_data = res_data.sort(['cycle','D24_int'], descending=False)

In [161]:
res_data.head(10)

shape: (10, 274)
┌────────────┬────────────┬─────────┬─────┬───┬─────────────┬─────────────┬─────────────┬───────┐
│ D141       ┆ D22        ┆ D24_int ┆ D44 ┆ … ┆ D135_110_dv ┆ D135_111_dv ┆ D135_112_dv ┆ cycle │
│ ---        ┆ ---        ┆ ---     ┆ --- ┆   ┆ ---         ┆ ---         ┆ ---         ┆ ---   │
│ f64        ┆ f32        ┆ f64     ┆ f64 ┆   ┆ f32         ┆ f32         ┆ f32         ┆ i32   │
╞════════════╪════════════╪═════════╪═════╪═══╪═════════════╪═════════════╪═════════════╪═══════╡
│ 497.043478 ┆ 20.706522  ┆ 20.0    ┆ 0.0 ┆ … ┆ 46.0        ┆ 45.0        ┆ 45.0        ┆ 1     │
│ 499.933036 ┆ 74.985718  ┆ 21.0    ┆ 0.0 ┆ … ┆ 69.0        ┆ 68.0        ┆ 69.0        ┆ 1     │
│ 487.54717  ┆ 173.903778 ┆ 22.0    ┆ 0.0 ┆ … ┆ 78.0        ┆ 75.0        ┆ 75.0        ┆ 1     │
│ 477.333333 ┆ 96.369301  ┆ 23.0    ┆ 0.0 ┆ … ┆ -74.0       ┆ -72.0       ┆ -73.0       ┆ 1     │
│ …          ┆ …          ┆ …       ┆ …   ┆ … ┆ …           ┆ …           ┆ …           ┆ …     │
│ 430.859649 ┆ 186.001755 ┆ 26.0    ┆ 0.0 ┆ … ┆ -10.0       ┆ -9.0        ┆ -11.0       ┆ 1     │
│ 409.671053 ┆ 183.076309 ┆ 27.0    ┆ 0.0 ┆ … ┆ 33.0        ┆ 34.0        ┆ 33.0        ┆ 1     │
│ 390.470588 ┆ 158.868622 ┆ 28.0    ┆ 0.0 ┆ … ┆ -74.0       ┆ -72.0       ┆ -73.0       ┆ 1     │
│ 379.466667 ┆ 154.79834  ┆ 29.0    ┆ 0.0 ┆ … ┆ -27.0       ┆ -27.0       ┆ -27.0       ┆ 1     │
└────────────┴────────────┴─────────┴─────┴───┴─────────────┴─────────────┴─────────────┴───────┘

In [164]:
df = pl.read_parquet('processed_data/D3_3.parquet')
df.head()

shape: (5, 274)
┌────────────┬───────────┬─────────┬─────┬───┬─────────────┬─────────────┬─────────────┬───────┐
│ D141       ┆ D22       ┆ D24_int ┆ D44 ┆ … ┆ D135_110_dv ┆ D135_111_dv ┆ D135_112_dv ┆ cycle │
│ ---        ┆ ---       ┆ ---     ┆ --- ┆   ┆ ---         ┆ ---         ┆ ---         ┆ ---   │
│ f64        ┆ f32       ┆ f64     ┆ f64 ┆   ┆ f32         ┆ f32         ┆ f32         ┆ i32   │
╞════════════╪═══════════╪═════════╪═════╪═══╪═════════════╪═════════════╪═════════════╪═══════╡
│ 420.467091 ┆ 8.671762  ┆ 60.0    ┆ 0.0 ┆ … ┆ -2.0        ┆ -2.0        ┆ -2.0        ┆ 1     │
│ 421.984733 ┆ 8.987022  ┆ 61.0    ┆ 0.0 ┆ … ┆ -2.0        ┆ -2.0        ┆ -2.0        ┆ 1     │
│ 420.342308 ┆ 5.921923  ┆ 44.0    ┆ 0.0 ┆ … ┆ 111.0       ┆ 109.0       ┆ 112.0       ┆ 2     │
│ 407.357602 ┆ 25.350107 ┆ 45.0    ┆ 0.0 ┆ … ┆ -72.0       ┆ -70.0       ┆ -74.0       ┆ 2     │
│ 440.396215 ┆ 8.752927  ┆ 46.0    ┆ 0.0 ┆ … ┆ -17.0       ┆ -18.0       ┆ -18.0       ┆ 2     │
└────────────┴───────────┴─────────┴─────┴───┴─────────────┴─────────────┴─────────────┴───────┘